In [73]:
import scipy.stats as ss
import scipy.special
from scipy import optimize
from mpmath import gamma
from scipy.stats import t as t_student
import numpy as np
import scipy
from scipy.stats import norm
from numpy import linalg as la
from scipy import sparse
from scipy.sparse.linalg import spsolve
import pandas as pd
from scipy.stats import multivariate_normal
import warnings
warnings.filterwarnings("ignore")

In [97]:
def covariance_matrix(sigma, rho):
  """Compute the covariance matrix.
  Args:
  - sigma (array): Array of volatilities of each stock.
  - rho (array): Correlation matrix.
  Returns:
  - SIGMA (array): Covariance matrix.
  """
  sigma = np.diag(sigma)  # Diagonal matrix of volatilities
  SIGMA = np.dot(sigma, np.dot(rho, sigma))  # Covariance matrix calculation
  return SIGMA

def VG_characteristic_function(u, SIGMA, T, r, theta, nu):
  """Calculate the characteristic function of Variance-Gamma process.
  Args:
  - u (array): Vector in Rd.
  - SIGMA (array): Covariance matrix.
  - T (float): Terminal time.
  - r (float): Short rate.
  - theta (array): Array of theta values.
  - nu (float): Nu parameter.
  Returns:
  - phi (complex): Characteristic function value.
  """
  d = len(theta)  # Number of stocks
  w = (1/nu) * np.log(1 - nu * theta - 0.5 * nu * np.diag(SIGMA))  # Martingale correction term
  phi = np.exp(np.multiply(1j * T, np.dot(r + w, u))) * (1 - np.multiply(1j * nu, np.dot(theta, u)) +
                                                          0.5 * nu * np.dot(u, np.dot(SIGMA, u))) ** (-T/nu)
  return phi

def fourier_payoff_call_on_min(u):
  """Compute the Fourier of the payoff of scaled (K = 1) call on min option.
  Args:
  - u (array): Array of Fourier frequencies.
  Returns:
  - payoff (float): Call on min option payoff Fourier transofrm value.
  """
  denominator = (np.multiply(1j, np.sum(u)) - 1) * np.prod(np.multiply(1j, u))
  return 1 / denominator

def fourier_payoff_basket_put(u):
  """Compute the Fourier of the payoff of scaled (K = 1) basket put option.
  Args:
  - u (array): Array of Fourier frequencies.
  Returns:
  - payoff (float): Call on min option payoff Fourier transofrm value.
  """
  numerator = np.prod(scipy.special.gamma(np.multiply(-1j,u)))
  denominator = scipy.special.gamma(-1j*(np.sum(u))+2)
  return (numerator/denominator)


def integrand_to_optimize_VG_call_on_min(R):
  """Calculate the integrand for QMC estimation of the rainbow option under VG model.
  Args:
  - R (array): Array of damping parameters.
  Returns:
  - integrand (float): Integrand value.
  """
  d = len(S0)  # Dimensionality
  X0 = np.log(np.divide(S0, K))  # Element-wise division
  y = np.multiply(1j, R)
  phi = VG_characteristic_function(y, SIGMA, T, r, theta, nu)  # Characteristic function
  p = fourier_payoff_call_on_min(y)  # Fourier Transformed Payoff function
  discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-np.dot(R, X0))  # Modified discount factor
  integrand = K * discount * phi * p
  return integrand

def integrand_to_optimize_VG_basket_put(R):
  """Calculate the integrand for QMC estimation of the basket put option under VG model.
  Args:
  - R (array): Array of damping parameters.
  Returns:
  - integrand (float): Integrand value.
  """
  d = len(S0)  # Dimensionality
  X0 = np.log(np.divide(S0, d*K))  # Element-wise division
  y = np.multiply(1j, R)
  phi = VG_characteristic_function(y, SIGMA, T, r, theta, nu)  # Characteristic function
  p = fourier_payoff_basket_put(y)  # Fourier Transformed Payoff function
  discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-np.dot(R, X0))  # Modified discount factor
  integrand = K * discount * phi * p
  return integrand

def t_student_pdf(x, sigma_IS):
    """
    Evaluate the probability density function (PDF) of the t-student distribution.
    Args:
    - x (array): Values at which to evaluate the PDF.
    - sigma_IS (float): Scale parameter for the t-student distribution.
    Returns:
    - pdf_values (array): PDF values corresponding to the input x.
    """
    return t_student.pdf(x=x, df=nu_IS, loc=0, scale=sigma_IS)


def t_student_ppf(x, sigma_IS):
    """
    Evaluate the percent point function (PPF) of the t-student distribution.
    Args:
    - x (array): Quantiles at which to evaluate the PPF.
    - sigma_IS (float): Scale parameter for the t-student distribution.
    Returns:
    - ppf_values (array): PPF values corresponding to the input x.
    """
    return t_student.ppf(q=x, df=nu_IS, loc=0, scale=sigma_IS)


def multivariate_t_student_pdf(u, nu_IS, SIGMA_IS, SIGMA_IS_inv):
  """
  Calculate the probability density function (PDF) of the multivariate t-student distribution.
  Parameters:
  - u (array): Array of Fourier frequencies.
  - nu_IS (float): Degrees of freedom for the t-student distribution.
  - SIGMA_IS (array): Covariance matrix.
  - SIGMA_IS_inv (array): Inverse of the covariance matrix.
  Returns:
  - pdf (float): Probability density function value.
  """
  d = len(u)  # Dimension of the random variable
  pdf = scipy.special.gamma(0.5 * (nu_IS + d)) / (scipy.special.gamma(0.5 * nu_IS) * (np.pi * nu_IS)**(0.5 * d) * np.sqrt(la.det(SIGMA_IS))) * (1 + (1 / nu_IS) * (u @ SIGMA_IS_inv @ u))**(-0.5 * (nu_IS + d))
  return pdf

def fourier_MC_call_on_min_VG_pricer(S0, K, r, T, sigma, rho, theta, nu, SIGMA, N, R, nu_IS, SIGMA_IS, alpha_conf, seed):
    """
    Estimate the price of basket put options under variance gamma using Monte Carlo simulation in Fourier space with importance sampling using Gaussian distribution.

    Args:
    - S0 (array): Vector of initial stock prices.
    - K (float): Strike price.
    - r (float): Risk-free interest rate.
    - T (float): Time to maturity.
    - sigma (array): Volatilities of each stock.
    - rho (array): Correlation matrix of Brownian motions.
    - theta (float): parameter of the variance gamma process.
    - nu (float): parameter of the variance gamma process.
    - SIGMA (array): Covariance matrix of Brownian motions.
    - N (int): Number of Monte Carlo samples.
    - R (array): Vector of damping parameters.
    - nu_IS (float): Degrees of freedom for the t-student distribution.
    - SIGMA_IS (array): Covariance matrix for the t-student distribution.
    - alpha_conf (float): Confidence level.
    - seed (int): Seed for the random generator.

    Returns:
    - MC_estimate (float): Estimated price of the rainbow option.
    - MC_stat_error (float): Statistical error of the Monte Carlo estimation.
    """
    np.random.seed(seed)
    # Number of stocks
    dimension = len(S0)
    # Logarithm of the element-wise division
    X0 = np.log(np.divide(S0, K))
    # Modified discount factor
    discount = ((2 * np.pi) ** (-dimension)) * np.exp(-r * T) * np.exp(-R @ X0)
    # Contains Monte Carlo price estimates
    V_list = np.zeros(N)
    L_IS = la.cholesky(SIGMA_IS)
    SIGMA_IS_inv = la.inv(SIGMA_IS)
    Z = np.random.multivariate_normal(mean=np.zeros(dimension), cov=np.identity(dimension), size=N)  # Independent samples from the multivariate standard normal distribution
    chi2 = np.random.chisquare(df=nu_IS, size=N)  # N samples from the chi-squared distribution
    t_student_samples = np.sqrt(nu_IS / chi2.reshape(N, 1)) * (L_IS @ (Z.T)).T  # N samples from the multivariate t-student distribution
    # For each sample
    for n in range(N):
      u = t_student_samples[n]  # Sample from the standard normal distribution
      y = u + np.multiply(1j, R)  # Shifting contour of integration by the damping parameters
      phi = VG_characteristic_function(y, SIGMA, T, r, theta, nu)  # Evaluate characteristic function
      # Evaluate Fourier Transformed Payoff function
      p = fourier_payoff_call_on_min(y)
      # Evaluate the multivariate PDF
      PDF_eval = multivariate_t_student_pdf(u, nu_IS, SIGMA_IS,SIGMA_IS_inv)
      # Compute Monte Carlo estimators
      V_list[n] = np.real(K * discount * np.exp(1j * u @ X0) * phi * p / PDF_eval)
    # Compute the Monte Carlo estimate
    MC_estimate = np.mean(V_list)
    # Compute the statistical error
    C_alpha = norm.ppf(1 - alpha_conf / 2)
    MC_stat_error = 1.96 * np.std(V_list) / np.sqrt(N)
    return MC_estimate, MC_stat_error

def fourier_MC_basket_put_VG_pricer(S0, K, r, T, sigma, rho, theta, nu, SIGMA, N, R, nu_IS, SIGMA_IS, alpha_conf, seed):
    """
    Estimate the price of basket put options under variance gamma using Monte Carlo simulation in Fourier space with importance sampling using Gaussian distribution.

    Args:
    - S0 (array): Vector of initial stock prices.
    - K (float): Strike price.
    - r (float): Risk-free interest rate.
    - T (float): Time to maturity.
    - sigma (array): Volatilities of each stock.
    - rho (array): Correlation matrix of Brownian motions.
    - theta (float): parameter of the variance gamma process.
    - nu (float): parameter of the variance gamma process.
    - SIGMA (array): Covariance matrix of Brownian motions.
    - N (int): Number of Monte Carlo samples.
    - R (array): Vector of damping parameters.
    - nu_IS (float): Degrees of freedom for the t-student distribution.
    - SIGMA_IS (array): Covariance matrix for the t-student distribution.
    - alpha_conf (float): Confidence level.
    - seed (int): Seed for the random generator.

    Returns:
    - MC_estimate (float): Estimated price of the rainbow option.
    - MC_stat_error (float): Statistical error of the Monte Carlo estimation.
    """
    np.random.seed(seed)
    # Number of stocks
    dimension = len(S0)
    # Logarithm of the element-wise division
    X0 = np.log(np.divide(S0, dimension * K))
    # Modified discount factor
    discount = ((2 * np.pi) ** (-dimension)) * np.exp(-r * T) * np.exp(-R @ X0)
    # Contains Monte Carlo price estimates
    V_list = np.zeros(N)
    L_IS = la.cholesky(SIGMA_IS)
    SIGMA_IS_inv = la.inv(SIGMA_IS)
    Z = np.random.multivariate_normal(mean=np.zeros(dimension), cov=np.identity(dimension), size=N)  # Independent samples from the multivariate standard normal distribution
    chi2 = np.random.chisquare(df=nu_IS, size=N)  # N samples from the chi-squared distribution
    t_student_samples = np.sqrt(nu_IS / chi2.reshape(N, 1)) * (L_IS @ (Z.T)).T  # N samples from the multivariate t-student distribution
    # For each sample
    for n in range(N):
      u = t_student_samples[n]  # Sample from the standard normal distribution
      y = u + np.multiply(1j, R)  # Shifting contour of integration by the damping parameters
      phi = VG_characteristic_function(y, SIGMA, T, r, theta, nu)  # Evaluate characteristic function
      # Evaluate Fourier Transformed Payoff function
      p = fourier_payoff_basket_put(y)
      # Evaluate the multivariate PDF
      PDF_eval = multivariate_t_student_pdf(u, nu_IS, SIGMA_IS,SIGMA_IS_inv)
      # Compute Monte Carlo estimators
      V_list[n] = np.real(K * discount * np.exp(1j * u @ X0) * phi * p / PDF_eval)
    # Compute the Monte Carlo estimate
    MC_estimate = np.mean(V_list)
    # Compute the statistical error
    C_alpha = norm.ppf(1 - alpha_conf / 2)
    MC_stat_error = 1.96 * np.std(V_list) / np.sqrt(N)
    return MC_estimate, MC_stat_error

# Pricing call on min options using MC in the Fourier domain

In [92]:
############### Model and payoff parameters ###############
######## Payoff Parameters ########
dimension = 4
S0 = 100*np.ones(dimension)
K = 100
r = 0
T = 1
######## Model Parameters ########
rho = np.identity(dimension)
sigma = 0.4*np.ones(dimension)
theta = -0.3*np.ones(dimension)
SIGMA = covariance_matrix(sigma,rho)
nu = 0.1

############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def call_on_min_constraint_1(R):
    return -1 * R

def call_on_min_constraint_2(R):
    return -1 - np.sum(R)

def VG_constraint(R):
    return 1 + nu * theta @ R - 0.5 * nu * R @ SIGMA @ R

cons = ( {'type': 'ineq', 'fun': VG_constraint},
        {'type': 'ineq', 'fun': call_on_min_constraint_1},
        {'type': 'ineq', 'fun': call_on_min_constraint_2},)

# Characteristic function of GBM is an entire function hence there are no related constraints to it.
R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_VG_call_on_min, constraints = cons, x0 = R_init , method = "Nelder-Mead" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

Optimal damping parameters: [-3.12869036 -3.1287065  -3.12874344 -3.12876568]


In [96]:
############### Model and payoff parameters ###############
######## Payoff Parameters ########
dimension = 4
S0 = 100*np.ones(dimension)
K = 100
r = 0
T = 1
######## Model Parameters ########
rho = np.identity(dimension)
sigma = 0.4*np.ones(dimension)
theta = -0.3*np.ones(dimension)
SIGMA = covariance_matrix(sigma,rho)
nu = 0.1
############### MC parameters ###############
N = 10**5 # number of MC sample paths
alpha_conf = 0.05 # confidence level for MC statistical error estimation
seed = 100 # random seed for reproducibility of results.
nu_IS = 2*T / nu - dimension #dimension
SIGMA_IS  = la.inv(SIGMA)
MC_Fourier_price_estimate, MC_Fourier_stat_error = fourier_MC_call_on_min_VG_pricer(S0, K, r, T, sigma, rho, theta, nu, SIGMA, N, R, nu_IS, SIGMA_IS, alpha_conf, seed)
print("MC price estimate =", round(MC_Fourier_price_estimate, 5),"\nMC relative statistical error = ", round(MC_Fourier_stat_error / MC_Fourier_price_estimate,4) )

MC price estimate = 0.41685 
MC relative statistical error =  0.0018


# Pricing basket put options using MC in the Fourier domain

In [79]:
############### Model and payoff parameters ###############
######## Payoff Parameters ########
dimension = 4
S0 = 100*np.ones(dimension)
K = 100
r = 0
T = 1
######## Model Parameters ########
rho = np.identity(dimension)
sigma = 0.4*np.ones(dimension)
theta = -0.3*np.ones(dimension)
SIGMA = covariance_matrix(sigma,rho)
nu = 0.2
############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def basket_put_constraint(R):
    return R

def VG_constraint(R):
    return 1 + nu * theta @ R - 0.5 * nu * R @ SIGMA @ R

cons = ( {'type': 'ineq', 'fun': VG_constraint},
      {'type': 'ineq', 'fun': basket_put_constraint},)

# Characteristic function of GBM is an entire function hence there are no related constraints to it.
R_init = 1*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_VG_basket_put, constraints = cons, x0 = R_init , method = "Nelder-Mead" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

Optimal damping parameters: [1.31748796 1.31751426 1.31757511 1.31751932]


In [86]:
############### Model and payoff parameters ###############
######## Payoff Parameters ########
dimension = 4
S0 = 100*np.ones(dimension)
K = 100
r = 0
T = 1
######## Model Parameters ########
rho = np.identity(dimension)
sigma = 0.4*np.ones(dimension)
theta = -0.3*np.ones(dimension)
SIGMA = covariance_matrix(sigma,rho)
nu = 0.1
############### MC parameters ###############
N = 10**5 # number of MC sample paths
alpha_conf = 0.05 # confidence level for MC statistical error estimation
seed = 100 # random seed for reproducibility of results.
nu_IS = 2*T / nu - dimension #dimension
SIGMA_IS  = la.inv(SIGMA)
MC_Fourier_price_estimate, MC_Fourier_stat_error = fourier_MC_basket_put_VG_pricer(S0, K, r, T, sigma, rho, theta, nu, SIGMA, N, R, nu_IS, SIGMA_IS, alpha_conf, seed)
print("MC price estimate =", round(MC_Fourier_price_estimate, 5),"\nMC relative statistical error = ", round(MC_Fourier_stat_error / MC_Fourier_price_estimate,4) )

MC price estimate = 8.54915 
MC relative statistical error =  0.0156
